# Unvariate Kalman filter

# Purpose
* implementation of 1D linear kalman filter inspired by: [Kalman-and-Bayesian-Filters-in-Python/blob/master/04-One-Dimensional-Kalman-Filters](https://github.com/rlabbe/Kalman-and-Bayesian-Filters-in-Python/blob/master/04-One-Dimensional-Kalman-Filters.ipynb)

# Methodology
* Implement a kalman filter to chase a ship at steady velocity

# Setup

In [ ]:
# %load imports.py

%matplotlib inline
%load_ext autoreload
%autoreload 2

import pandas as pd
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option("display.max_columns", None)
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
import filterpy.stats as stats

In [ ]:
from collections import namedtuple
gaussian = namedtuple('Gaussian', ['mean', 'var'])
gaussian.__repr__ = lambda s: '𝒩(μ={:.3f}, 𝜎²={:.3f})'.format(s[0], s[1])

In [ ]:
gaussian(mean=0, var=1)

What is the sum of two Gaussians? In the last chapter I proved that:

$$\begin{gathered}
\mu = \mu_1 + \mu_2 \\
\sigma^2 = \sigma^2_1 + \sigma^2_2
\end{gathered}$$

In [ ]:
def predict(pos, movement):
    return gaussian(pos.mean + movement.mean, pos.var + movement.var)

In [ ]:
def gaussian_multiply(g1, g2):
    mean = (g1.var * g2.mean + g2.var * g1.mean) / (g1.var + g2.var)
    variance = (g1.var * g2.var) / (g1.var + g2.var)
    return gaussian(mean, variance)

def update(prior, likelihood):
    posterior = gaussian_multiply(likelihood, prior)
    return posterior

In [ ]:
z = gaussian(10., 1.)  # Gaussian N(10, 1)

product = gaussian_multiply(z, z)

xs = np.arange(5, 15, 0.1)
ys = [stats.gaussian(x, z.mean, z.var) for x in xs]
plt.plot(xs, ys, label='$\mathcal{N}(10,1)$')

ys = [stats.gaussian(x, product.mean, product.var) for x in xs]
plt.plot(xs, ys, label='$\mathcal{N}(10,1) \\times \mathcal{N}(10,1)$', ls='--')
plt.legend()
print(product)

In [ ]:
z1 = gaussian(-3., 1)  # Gaussian N(10, 1)
z2 = gaussian(3., 1)  # Gaussian N(10, 1)

product = gaussian_multiply(z1, z2)

xs = np.arange(-6, 6, 0.1)

ys = [stats.gaussian(x, z1.mean, z1.var) for x in xs]
plt.plot(xs, ys, label='$\mathcal{N}(-3,1)$')

ys = [stats.gaussian(x, z2.mean, z2.var) for x in xs]
plt.plot(xs, ys, label='$\mathcal{N}(3,1)$')


ys = [stats.gaussian(x, product.mean, product.var) for x in xs]
plt.plot(xs, ys, label='$\mathcal{N}(10,1) \\times \mathcal{N}(10,1)$', ls='--')
plt.legend()
print(product)

In [ ]:
v = 5.5
N=7
t = np.linspace(0,10,N)
dt = t[1]-t[0]
x_real = v*t

np.random.seed(42)

sensor_var = 3**2
epsilon = np.random.normal(scale=np.sqrt(sensor_var), size=N)

process_var = 1**2
w = np.random.normal(scale=np.sqrt(process_var), size=N)

zs = x_real + epsilon*dt + w*dt

## Kalman filtering

In [ ]:
def kalman(x0, process_model, zs, sensor_var = 5**2):
    
    prior = x0
    
    xs = np.zeros((N, 2))
    priors = np.zeros((N, 2))
    for i, z in enumerate(zs):
         
        z_ = gaussian(z, sensor_var)
        x = update(prior=prior, likelihood=z_)
        
        # Save
        priors[i] = prior
        xs[i] = x
        
        prior = predict(pos=x, movement=process_model)   
        
    return priors, xs

In [ ]:
process_model = gaussian(v*dt, process_var)
process_model

In [ ]:
x0 = gaussian(0., sensor_var**2)
process_model = gaussian(3.0*v*dt, process_var)
priors, xs = kalman(x0=x0, process_model=process_model, zs=zs, sensor_var=sensor_var)

In [ ]:
fig,ax=plt.subplots()

ax.plot(t, x_real, '-', label='real')
ax.plot(t, zs, 'o', label='measurement')
ax.plot(t, xs[:,0], '-', label='filter')
ax.plot(t, priors[:,0], '--', label='prior')
ax.legend()


fig,ax=plt.subplots()
ax.plot([t[0],t[-1]], [sensor_var, sensor_var], label='measurement')
ax.plot(t, xs[:,1], label='filter')
ax.plot(t, priors[:,1], label='prior')
ax.set_title('Variances')
ax.set_ylim(0,50)
ax.legend();



In [ ]:
def plot_filter(xs):


    fig,ax=plt.subplots()
    
    likelihoods = []
    for x in xs:
    
        mu = x[0]
        var = x[1]
        sigma = np.sqrt(var)
            
        rv = stats.multivariate_normal(mean=mu, cov=[var])
        likelihoods.append(rv.pdf(mu))
        
        xs_ = np.linspace(mu-3*sigma, mu+3*sigma,100)
        ys_ = rv.pdf(xs_)
            
        ax.plot(xs_, ys_, ls='-')
        
    ax.plot(xs[:,0], likelihoods)
        
    ax.set_xlabel(r'$\hat{x}$')
    ax.set_ylabel('density');



In [ ]:
plot_filter(xs)

In [ ]:
plot_filter(priors)

In [ ]:
process_model = gaussian(v*dt, process_var)
priors, xs = kalman(x0=x0, process_model=process_model, zs=zs, sensor_var=sensor_var)
plot_filter(priors)

In [ ]:
process_model = gaussian(2*v*dt, process_var)
priors, xs = kalman(x0=x0, process_model=process_model, zs=zs, sensor_var=sensor_var)
plot_filter(priors)

In [ ]:
def loglikelihood(priors,xs):
    
    loglikelihood = 0
    
    #for x, prior in zip(xs,priors):
    #    
    #    mu = x[0]
    #    var = x[1]
    #    rv = stats.multivariate_normal(mean=mu, cov=[var])
    #    
    #    loglikelihood+=rv.logpdf(prior[0])
    
    
    var = xs[-1,1]
        
    for x, prior in zip(xs,priors):
        
        mu = x[0]
        rv = stats.multivariate_normal(mean=mu, cov=[var])
        
        loglikelihood+=rv.logpdf(prior[0])
        
    return loglikelihood
    
    

In [ ]:
vs = np.linspace(0.5*v,1.5*v,20)
df_variation = pd.DataFrame(index=vs)

for v_ in vs:

    process_model = gaussian(v_*dt, process_var)
    priors, xs = kalman(x0=x0, process_model=process_model, zs=zs, sensor_var=sensor_var)
    
    df_variation.loc[v_,'loglikelihood'] = loglikelihood(priors, xs)
    

In [ ]:
fig,ax=plt.subplots()

df_variation.plot(y='loglikelihood', ax=ax)
ax.plot([v,v],[df_variation['loglikelihood'].min(),df_variation['loglikelihood'].max()],'r--', label='v (real)');
ax.legend();